## Ejercicio 1

Consulta los segmentos definidos en el ejecutable generado con el comando readelf -l  y completa la siguiente tabla. 
- Los segmentos de la tabla corresponden a las secciones de tipo LOAD. La primera de ellas PHDR es la 00, INTERP la 01 y así sucesivamente. 
- Algunas secciones del programa pueden estar asignadas al mismo segmento del ejecutable.

| Segmento  | código C correspondiente  | Offset    | dir.virtual         | Flags |
|---        |---                        |---        |---                  |---    |
| .text     | Codigo del script         | 0x0..1000 | 0x0000 .. 0000 1000 | RE |
| .rodata   | const char *msg="El re... | 0x0..2000 | 0x0000 .. 0000 2000 | R  |
| .bss      | int num=22; sta int fac=2 | 0x0..2db0 | 0x0000 .. 0000 3db0 | RW |
| .data     | int mul;                  | 0x0..2db0 | 0x0000 .. 0000 3db0 | RW |

Ejecutar ahora el programa y obtener los segmentos de memoria virtual accediendo al fichero maps del directorio del proceso en /proc. Nota: Se puede identificar la correspondencia de los 
 
Sistemas Operativos 1 
 
segmentos del ejecutable y los segmentos de memoria virtual comparando el offset y los flags: 

| Segmento  | rango de direcciones virtuales  | Offset del fichero   | Flags | Tipo | Ruta del fichero |
|---        |---                              |---              |---           |---    | ---            |
| .text     | 55d963ev7000 - 55d963eb8000 | 00001000 | r-xp | map       | /home/ubuntu/hoja4/ejercicio1 |
| .rodata   | 55d963ev8000 - 55d963eba000 | 00002000 | r--p | map       | /home/ubuntu/hoja4/ejercicio1 |
| .bss      | 55d963eva000 - 55d963ebb000 | 00003000 | rw-p | map       | /home/ubuntu/hoja4/ejercicio1 |
| .data     | 55d963eva000 - 55d963ebb000 | 00003000 | rw-p | map       | /home/ubuntu/hoja4/ejercicio1 |
| [heap]    | 55d964922000 - 55da0be81000 | 00000000 | rw-p | anonimo   | [heap]                        |
| [stack]   | 7ffd02ab0000 - 7ffd02bb1000 | 00000000 | rw-p | anonimo   | [stack]                       |

Cuestiones: 
- En qué segmento(s) de memoria virtual está la cadena "El resultado es:\n". El fichero proyectado del segmento de memoria virtual se puede acceder desde el directorio del proceso /proc/<pid proceso>/map_files/<rango del segmento> ¿Qué tipo de ficheros son?. 

En el .rodata ya que es un literal, esta marcado como solo lectura (const) y no cambia durante la ejecución.

- En la salida del comando readelf se muestra el inicio del programa (entry point). ¿En qué segmento del fichero está ubicado? ¿Y en qué segmento de memoria virtual? 

Se encuentra en el segmento load del elf que contiene el .text
En memoria virtual se ubica en el mismo segmento del codigo

- Dibuja esquemáticamente (como en las transparencias de teoría) el espacio de direcciones del proceso con las regiones correspondientes. 

Direcciones altas
+----------------------+
|      [stack]         |
+----------------------+
|      [vvar]          |
+----------------------+
|      [vdso]          |
+----------------------+
|  mmap anónimo        |
+----------------------+
|  Bibliotecas (libc)  |
+----------------------+
|        [heap]        |
+----------------------+
|        .bss          |
+----------------------+
|        .data         |
+----------------------+
|       .rodata        |
+----------------------+
|        .text         |
+----------------------+
Direcciones bajas

- Investiga cuál es el contenido y propósito de las áreas de memoria [vdso] y [vvar]. 

[vdso] Virtual elf dynamic shared object. Es donde esta el codigo optimizado para llamadas al kernel.
[vvar] Virtual variables. Datos auxiliares para vdso.

## Ejercicio 3
| Dirección inicial | Direccion Final | Offset | Flags |
|---|---|---|---|
| 7f84560b8000 | 7f84560b9000 | 000000000 | rw-p |

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <string.h>
#include <unistd.h>

int main(void){
	// Reservamos en memoria el espacio 1024
	int *mem = mmap(NULL, 1024, 
			PROT_READ | PROT_WRITE,
			MAP_PRIVATE | MAP_ANONYMOUS,
			-1,	// descriptor a -1 para regiones anonimas
			0);
	if (mem == MAP_FAILED){
		perror("mmap");
		return 1;
	}
	// asignamos \0 con memset
	memset(mem, '\0', 1024);

	printf("PID:%i Direccion del segmento: %p\n", getpid(), mem);
	sleep(600);
	return 0;
}


## Ejercicio 4
Re-escribir el ejercicio 11 de la Hoja 3 proyectando el fichero de salida (output.txt) 
en la memoria virtual

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <string.h>
#include <sys/mman.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <sys/wait.h>

#define HIJOS 5

void escribe(int n_hijo){
	int fd = open("./output.txt", O_RDWR);
	// Ha habido un error abriendo el archivo
	if(fd == -1) return;
	// mappeamos la memoria
	char *map = mmap(NULL, HIJOS*(HIJOS+1),
			PROT_WRITE,
			MAP_SHARED,
			fd, 0);
	if(map == MAP_FAILED){
		perror("mmap");
		close(fd);
	}
	// Antes de escribir desplazamos la escritura (5 ceros + (n_hijo-1 *5)
	off_t off= (HIJOS+1) * n_hijo;
	// rellenamos con el caracterx
	memset(map + off, '0' + n_hijo, HIJOS);
	close(fd);

	if(msync(map, HIJOS * (HIJOS +1), MS_SYNC) == -1){
		perror("msync");
	}
	munmap(map, HIJOS*(HIJOS + 1));
}

int main(int argc, char *argv[]){
	// escribimos los 0 iniciales
	int fd = open("./output.txt", O_RDWR | O_CREAT | O_TRUNC , 0644);
	// asignamos memoria con ftruncate
	int res = ftruncate(fd, HIJOS*(HIJOS+1));
	if(res == -1){
		perror("ftruncate");
		return 1;
	}
	// mappeamos la memoria
	char *map = mmap(NULL, HIJOS*(HIJOS+1),
			PROT_WRITE,
			MAP_SHARED,
			fd, 0);
	if(map == MAP_FAILED){
		perror("mmap");
		close(fd);
		return 1;
	}
	// rellenamos con el caracterx
	memset(map, '0' , HIJOS+1);
	close(fd);
	for(int i = 0; i<HIJOS; i++){
		pid_t pid = fork();

		switch(pid){
		case -1:
			perror("error en fork");
			exit(1);
		case 0:
			printf("Hijo: %i\n", i);
			escribe(i);
			exit(i);
		default:
			printf("Padre\n");
			break;
		}
	}
	int status;
	for(int i=0; i<5;i++){
		pid_t pid = wait(&status);
		printf("Hijo %d con PID %d terminó\n", WEXITSTATUS(status), pid);
	}

	if(msync(map, HIJOS*(HIJOS +1), MS_SYNC) == -1){
		perror("msync");
	}
	munmap(map, HIJOS*(HIJOS + 1));
	return 0;
}

